In [1]:
#from time import time

from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import GridSearchCV

from my_fun import read_feature
from my_fun import run_model
from my_fun import summary_result
from my_fun import plot_result_mergecv

brain_feature_list, subject_info_merge = read_feature()

# remove samples with Med_rating higher than 1:
# subject_info_merge = subject_info_merge.loc[subject_info_merge["Med_Rating"]<2]
    
# subject_info_merge.to_csv("subject_info_merge.csv", sep = ",")




(165, 4)
(8719, 6)
cat_vbm:
(856, 143)
label fa:
(856, 50)
tract fa:
(856, 20)
tract md:
(856, 20)
label md:
(856, 50)
alff
(856, 246)
falff
(856, 246)
reho
(856, 246)


In [2]:
print("participants:")
print("number of subjects: %d" % subject_info_merge.shape[0])
print("meam  and std of age: %.2f %.2f" % (subject_info_merge["age_at_cnb"].mean(),\
                                           subject_info_merge["age_at_cnb"].std()))
print("range of age: %d - %d" % (subject_info_merge["age_at_cnb"].max(),\
                                 subject_info_merge["age_at_cnb"].min()))

print("number of females: %d" % subject_info_merge.loc[subject_info_merge['Sex_x']==0,:].shape[0])
print("number of males: %d" % subject_info_merge.loc[subject_info_merge['Sex_x']==1,:].shape[0])



participants:
number of subjects: 839
meam  and std of age: 14.37 3.37
range of age: 21 - 8
number of females: 451
number of males: 388


In [3]:
from sklearn.model_selection import KFold
# from sklearn.model_selection import StratifiedKFold

subject_info_hc = subject_info_merge

print(subject_info_hc.head())

#subject_info_hc = subject_info_merge.loc[subject_info_merge["ptsd"]==0]
# subject_info_ptsd = subject_info_merge.loc[subject_info_merge["ptsd"]==2]
# subject_info_trauma = subject_info_merge.loc[subject_info_merge["ptsd"]==1]

# count number of subjects in each group: (with brain imaging data:)
print(len(subject_info_hc.index))

# tune model in 3-fold CV:
# n fold CV on hc data:
nfold = 10
seed = 111
kf = KFold(n_splits=nfold, shuffle = True, random_state=seed)
# kf = StratifiedKFold(n_splits=nfold, shuffle = True, random_state=seed)


         SUBJID        Race  Sex_x  Med_birth_year  Med_Rating  age_at_cnb  \
0  600009963128          EA      0          2001.0         1.0         9.0   
1  600018902293          EA      0          1996.0         1.0        15.0   
2  600020927179          EA      0          1994.0         3.0        15.0   
3  600038720566      EA, AA      0          1999.0         1.0        11.0   
4  600039015048  EA, AA, HI      1          1999.0         1.0        11.0   

   Sex_y   age   ptsd         names  
0    NaN   NaN  999.0  600009963128  
1    NaN   NaN  999.0  600018902293  
2    NaN   NaN  999.0  600020927179  
3    NaN   NaN  999.0  600038720566  
4    2.0  10.0    0.0  600039015048  
839


In [4]:
############################################# DNN (pytorch) #######################################
import torch
import torch.nn.functional as F
import torch.nn as nn
from torch.autograd import Variable
from torch.optim.lr_scheduler import ReduceLROnPlateau

# from skorch import NeuralNetClassifier
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import StratifiedShuffleSplit
import matplotlib.pyplot as plt
from collections import OrderedDict

print(torch.__version__)
torch.manual_seed(999)    # reproducible
# https://discuss.pytorch.org/t/size-mismatch-beginner-troubles/16789
# model setup

def smooth(y, box_pts):
    box = np.ones(box_pts)/box_pts
    y_smooth = np.convolve(y, box, mode='same')
    return y_smooth


def weights_init(m):
    if isinstance(m, nn.Linear):
        print('initialize weight...')
        nn.init.xavier_uniform_(m.weight.data) 

def create_plot():
    fig = plt.figure(figsize=(8,8))
    ax = fig.add_subplot(111)
    plt.ion() #Turn the interactive mode on.
    fig.show()
    fig.canvas.draw()
    return(fig)
    
    
def update_plot(fig, plot_list, loss_info):
    
    ax = plt.gca()
    plt.ion() #Turn the interactive mode on.
    ax.clear()
    
    color = [['r--', 'b--'], ['g--', 'y--']]
    
    for i in range(len(plot_list)):
        i_list = plot_list[i]
        i_list_s = smooth(i_list, box_pts = 100)
        epoch = len(i_list)
        ax.plot(range(epoch), i_list, color[i][0], i_list_s, color[i][1])
    
    plt.ylim(0, 30) 
    plt.text(10, 30 , loss_info, fontsize=10)
    fig.canvas.draw()
    
def end_plot(fig, plot_list, loss_info, opt_loss_info, opt_epochs):
    plt.ion() #Turn the interactive mode on.
    ax = plt.gca()
    ax.clear()
    color = [['r--', 'b--'], ['g--', 'y--']]
    
    for i in range(len(plot_list)):
        i_list = plot_list[i]
        i_list_s = smooth(i_list, box_pts = 100)
        epoch = len(i_list)
        ax.plot(range(epoch), i_list, color[i][0], i_list_s, color[i][1])
        
    plt.ylim(0, 30) 
    plt.text(10, 30 , loss_info, fontsize=10)
    plt.text(10, 20 , opt_loss_info, fontsize=10)
    ax.axvline(opt_epochs)
    fig.canvas.draw()
    

def transform_variable(X, y):
    
    X_train_torch = Variable(torch.from_numpy(X))
    y_train_torch = Variable(torch.from_numpy(y))
    
    X_train_torch = X_train_torch.float()
    y_train_torch = y_train_torch.float()

    labels = y_train_torch.view(y.shape[0],1)
    
    return (X_train_torch, labels)


class Model(nn.Module): 
    
    def __init__(self, min_loss = .5, l_rate = .001, max_epochs = 10000):
        
        super(Model, self).__init__() 
        self.min_loss = min_loss
        self.l_rate = l_rate
        self.max_epochs = max_epochs
    
        
    def build_layer(self, input_dim):
        
        if input_dim<100:
            n_units = [input_dim, 60, 40, 20, 1]
        elif input_dim<200:
            #n_units = [input_dim, 100, 100, 50, 20, 1] 
            n_units = [input_dim, 100, 50, 30, 5, 1]
        elif input_dim<300:
            n_units = [input_dim, 200, 100, 50, 20, 1]      
        else:
            n_units = [input_dim, 400, 150, 50, 30, 5, 1]
        
        
        self.sigmoid = torch.nn.Sigmoid()
        self.n_layers = len(n_units)-1
        
        layers = OrderedDict()
        for i in range(self.n_layers):
            n_hidden = n_units[i]
            n_hidden2 = n_units[i+1]
            
            layers[str(i)] = nn.Linear(n_hidden, n_hidden2)
            
        self.layers = nn.Sequential(layers)  
        print("build layer:")
        #print(self)
        
        self.optimizer = torch.optim.SGD(self.parameters(), 
                                    lr = self.l_rate, 
                                    weight_decay=1e-3, 
                                    momentum=0.9, 
                                    dampening = 0,
                                    nesterov=True) #Stochastic Gradient Descent 
        
        #self.scheduler = torch.optim.lr_scheduler.StepLR(self.optimizer, step_size=500, gamma=.8)
        self.scheduler = torch.optim.lr_scheduler.MultiStepLR(self.optimizer, 
                                                              milestones=[500, 1000, 1500, 3000, 6000], 
                                                              gamma=0.5)
        
#         optimizer = torch.optim.Adagrad(self.parameters(), 
#                                         lr = self.l_rate, 
#                                         lr_decay = 1e-9,
#                                         weight_decay=1e-9,
#                                        )

        self.criterion = nn.MSELoss()# Mean Squared Loss
    
    def forward (self, x, **kwargs):

        for i in range(self.n_layers):
            #layer = eval('self.layer' + str(i+1))
            layer = self.layers[i]
            
            if i==0:
                # the 1st layer:
                out = F.relu(layer(x))
                #out = self.dropout(out)
            elif i<self.n_layers-2:
                # the middle layers use sigmoid.
                out = self.sigmoid(layer(out))
                #out = self.dropout(out)
            elif i==self.n_layers-2:
                out = F.relu(layer(out))
                #out = self.dropout(out)
            else:
                y_pred = layer(out) # linear output
            
        return y_pred
    
    
    def grad_decent(self, X_train_torch, labels):
        self.scheduler.step()
        #clear grads 
        self.optimizer.zero_grad()
        #forward to get predicted values
        outputs = self(X_train_torch) # model predict,  outputs = net.forward(inputs)

        loss = self.criterion(outputs, labels)
        loss.backward()# back props
        self.optimizer.step()# update the parameters 
        self.loss = loss
    
    
    def fit (self, X, y, ):
        """
        Fit model with all the input samples. It is recommended to first use fit_cv to determined the 
        optimized value of min_loss.
        """
        min_loss = self.min_loss
        max_epochs = self.max_epochs
        
        #torch.manual_seed(999)    # reproducible
        self.build_layer(input_dim = X.shape[1])

        torch.manual_seed(999) 
        self.apply(weights_init)
        print('training with all training data:')
        opt_loss_mean = -9999

        X_train_torch, labels = transform_variable(X, y)

        loss_list = []
        epoch = 0
        #fig = create_plot()
        while True:            
            #increase the number of epochs by 1 every time
            epoch +=1

            self.grad_decent(X_train_torch, labels)
            loss = self.loss.item()
            loss_list = np.append(loss_list, loss)

            if epoch % 100 == 0:
                loss_info = 'epoch %d, loss %.4f' % (epoch, loss)
                #update_plot(fig, [loss_list], loss_info)

            # define the mean loss to prevent overfitting.
            if loss<min_loss or epoch>max_epochs:
                print('stop with loss', loss)
                break
    
    
    def fit_cv (self, X_train, y_train, X_test, y_test):
        """
        Use test data to compute validation accuracy.
        """
        
        min_loss = self.min_loss
        max_epochs = self.max_epochs
        
        #torch.manual_seed(999)    # reproducible
        self.build_layer(input_dim = X.shape[1])
        X_train_torch, labels = transform_variable(X_train, y_train)

        # Collect errors to evaluate performance
        loss_list = []
        corr_list = []

        torch.manual_seed(999)
        #self.apply(weights_init)

        epoch = 0
        fig = create_plot()
        while True:

            #increase the number of epochs by 1 every time
            epoch +=1

            self.grad_decent(X_train_torch, labels)
            loss = self.loss.item()

            y_prediction = self.predict(X_test)

            corr = np.corrcoef(y_prediction, y_test)[0,1]
            corr_list = np.append(corr_list, corr) 
            loss_list = np.append(loss_list, loss)

            max_corr = np.amax(corr_list)

            if epoch % 100 == 0:
                loss_info = 'epoch %d, loss %.4f, test cor %.4f' % (epoch, loss, corr)
                update_plot(fig, [loss_list, corr_list*10], loss_info)

            if epoch>max_epochs or loss< min_loss:
                break

        corr_list_s = smooth(corr_list, box_pts = 100)
        opt_epochs = corr_list_s.argmax()
        opt_loss = loss_list[opt_epochs]

        print('max coor at epochs: %d, loss %.4f, with test cor: %.4f' % 
              (opt_epochs, opt_loss, corr_list[opt_epochs]))

        loss_info = 'epoch %d, loss %.4f, test cor %.4f' % (epoch, loss, corr)

        opt_loss_info = 'opt_epoch %d, opt_loss %.4f, test cor %.4f' % \
            (opt_epochs, opt_loss, corr_list[opt_epochs])

        update_plot(fig, [loss_list, corr_list*10], loss_info)
        end_plot(fig, [loss_list, corr_list*10], loss_info, opt_loss_info, opt_epochs)

        
        return(opt_loss)

    
    def predict(self, X_test):
        
        X_test_torch = Variable(torch.from_numpy(X_test))
        X_test_torch = X_test_torch.float()
        
        y_prediction = self(X_test_torch)
        y_prediction = y_prediction.detach().numpy().flatten()
        
        return y_prediction

0.4.1


In [5]:
import pandas as pd
%matplotlib notebook

# select only multi-modal features.
brain_feature = brain_feature_list[-1][0]
print(brain_feature_list[-1][1])

feature_importance = pd.DataFrame(index = brain_feature.columns[1:], 
                                  columns = ['performance reduction'])
all_feature_performance = 0.735594

i = 1
for i_rm_feature in brain_feature.columns[1:]:
    # we cannot use i_rm_feature directly due to duplicated column names
    brain_feature_sub = brain_feature.iloc[:, [j for j, c in enumerate(brain_feature.columns) if j != i]]
    #brain_feature_sub = brain_feature.drop(brain_feature.iloc[:, i], axis=1, inplace=False)
    
    print(brain_feature_sub.shape)
    brain_feature_sub_list = [[brain_feature_sub, 'Multi-modal']]
    print(i_rm_feature)

    # run DNN with default parameter:
    result_table_dnn, scatter_data_dnn = run_model(Model(), subject_info_hc, brain_feature_sub_list, kf, fit_method = 0)
    result_summary_dnn, result_plot_dnn = summary_result(result_table_dnn)
    
    feature_importance.iloc[i-1, 0] = \
        all_feature_performance - result_summary_dnn['R square', 'mean'].values
    i +=1

Multi-modal
(856, 774)
l3thVen
processing on: %s -------------------------- Multi-modal
run_model on CV: 1
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.494398295879364
run_model on CV: 2
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:


/home/xin/environments/lib/python3.6/site-packages/scipy/optimize/minpack.py:794: OptimizeWarning: Covariance of the parameters could not be estimated
  category=OptimizeWarning)


stop with loss 0.49943211674690247
run_model on CV: 3
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4944446086883545
run_model on CV: 4
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49530738592147827
run_model on CV: 5
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4997306168079376
run_model on CV: 6
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4991059899330139
run_model on CV: 7
build layer:
initialize weight...
initialize 

stop with loss 0.497001975774765
run_model on CV: 4
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4947796165943146
run_model on CV: 5
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49592843651771545
run_model on CV: 6
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4943509101867676
run_model on CV: 7
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49553489685058594
run_model on CV: 8
build layer:
initialize weight...
initialize w

stop with loss 0.494904488325119
run_model on CV: 5
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49922943115234375
run_model on CV: 6
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4989495277404785
run_model on CV: 7
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4952746629714966
run_model on CV: 8
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4957175850868225
run_model on CV: 9
build layer:
initialize weight...
initialize we

...

stop with loss 0.49816569685935974
run_model on CV: 5
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49844858050346375
run_model on CV: 6
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49994561076164246
run_model on CV: 7
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49612486362457275
run_model on CV: 8
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4964074492454529
run_model on CV: 9
build layer:
initialize weight...
initializ

stop with loss 0.49918946623802185
run_model on CV: 6
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49932560324668884
run_model on CV: 7
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4963519275188446
run_model on CV: 8
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4958062171936035
run_model on CV: 9
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4943212866783142
run_model on CV: 10
build layer:
initialize weight...
initialize

stop with loss 0.49440324306488037
run_model on CV: 7
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4965461194515228
run_model on CV: 8
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49459636211395264
run_model on CV: 9
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4958861470222473
run_model on CV: 10
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49477124214172363
summary_result:
            Pearson r            R square     

stop with loss 0.4947860538959503
run_model on CV: 8
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49977782368659973
run_model on CV: 9
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49690139293670654
run_model on CV: 10
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4980544149875641
summary_result:
            Pearson r          R square                 MAE            \
                 mean     std      mean       std      mean       std   
Multi-modal  0.861121  0.0293  0.742302  0.049499  1.328312  0.105645   

                 rmse            
                 m

stop with loss 0.4984707236289978
run_model on CV: 9
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.4937065839767456
run_model on CV: 10
build layer:
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
initialize weight...
training with all training data:
stop with loss 0.49443501234054565
summary_result:
            Pearson r            R square                 MAE            \
                 mean       std      mean       std      mean       std   
Multi-modal  0.861105  0.028774  0.742248  0.048668  1.328065  0.105473   

                 rmse            
                 mean       std  
Multi-modal  1.715158  0.127692  
(856, 774)
 Cluster.index=245
processing on: %s -------------------------- Multi-modal
run_model on CV: 1
build layer:
initialize weight...
initialize weight...
initializ

In [6]:
print(brain_feature)

            names  l3thVen  r3thVen  l4thVen  r4thVen    lAcc    rAcc    lAmy  \
0    600009963128   0.0960   0.0677   0.0735   0.0759  0.5301  0.5572  0.8959   
1    600018902293   0.0616   0.0539   0.0417   0.0545  0.3319  0.3289  0.7542   
2    600020927179   0.0514   0.0457   0.0399   0.0473  0.3592  0.3814  0.8929   
3    600038720566   0.0631   0.0521   0.0484   0.0518  0.4657  0.4629  0.7687   
4    600039015048   0.0658   0.0694   0.0515   0.0556  0.4251  0.4456  0.8340   
5    600039665619   0.0771   0.0624   0.0546   0.0500  0.4726  0.4839  0.9014   
6    600045443700   0.0446   0.0285   0.0372   0.0387  0.3804  0.3932  0.7517   
7    600052596653   0.0756   0.0570   0.0465   0.0521  0.4438  0.4692  0.9139   
8    600053476714   0.0649   0.0639   0.0361   0.0430  0.4679  0.4665  0.9099   
9    600054124128   0.0936   0.0615   0.0556   0.0540  0.5150  0.4685  0.8753   
10   600059829137   0.0708   0.0689   0.0561   0.0522  0.3897  0.4569  0.8919   
11   600061763148   0.0464  

In [7]:
feature_importance.to_csv("result/feature_importance.csv", sep = ",")

In [8]:

feature_list = ['gmv', 'fa', 'md', 'alff', 'reho']
feature_length = [0, 142, 70, 70, 246, 246]
feature_idx = np.cumsum(feature_length)

i = 0
for i_feature in feature_list:
    start_idx = feature_idx[i]
    end_idx = feature_idx[i+1]
    i +=1
    
    feature_importance_sub = feature_importance.iloc[start_idx:end_idx,:]
    print(feature_importance_sub)
    feature_importance_sub.to_csv("result/feature_importhance_" + i_feature + ".csv", sep = ",")

                        l3thVen   r3thVen   l4thVen   r4thVen      lAcc  \
performance reduction  0.008759  0.007752  0.006653  0.006847  0.006351   

                           rAcc      lAmy      rAmy      lBst      rBst  \
performance reduction  0.006307  0.006944  0.008016  0.007807  0.008399   

                              ...           Cluster.index=237  \
performance reduction         ...                   -0.006612   

                        Cluster.index=238   Cluster.index=239  \
performance reduction           -0.007043           -0.006779   

                        Cluster.index=240   Cluster.index=241  \
performance reduction           -0.007091           -0.006708   

                        Cluster.index=242   Cluster.index=243  \
performance reduction           -0.006849           -0.007509   

                        Cluster.index=244   Cluster.index=245  \
performance reduction           -0.006654           -0.006844   

                        Cluster.index=246  